In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("../input/source-based-news-classification/news_articles.csv")
df.head(3)

In [ ]:
df["type"].unique()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df=df.dropna() #Drop Nan Values

In [ ]:
df.reset_index(inplace=True)

In [ ]:
import seaborn as sns
print(df.label.value_counts())
sns.countplot(x=df["label"])

In [ ]:
df["label"]=df.label.map({'Real':1,'Fake':0})
print(df.label.unique())

In [ ]:
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.models import Sequential

In [ ]:
y=df['label']
X = df['title_without_stopwords'] + ' ' + df['text_without_stopwords']

In [ ]:
X = X.astype(str)

In [ ]:
# one hot representation
voc_size=10000
ohr=[one_hot(words,voc_size)for words in X]

In [ ]:
# Embedding Representation
sent_length=120
emb_rep=pad_sequences(ohr,padding='pre',maxlen=sent_length)
print(emb_rep)

In [ ]:
X=np.array(emb_rep)
y=np.array(y)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
# Predicting from test data
y_pred=model.predict_classes(X_test)

#Creating confusion matrix
cm = metrics.confusion_matrix(y_test, y_pred)
print("confusion_matrix: \n",cm)
print("accuracy_score: ",metrics.accuracy_score(y_test,y_pred))

In [ ]:
# Creating bidirectional lstm model
from tensorflow.keras.layers import Bidirectional, Dropout
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100))) # Bidirectional LSTM layer
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

In [ ]:
# Fitting the model
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

In [ ]:
y_pred1=model1.predict_classes(X_test)

#Confusion matrix
cm = metrics.confusion_matrix(y_test, y_pred1)
cm

In [ ]:
#Calculating Accuracy score
accuracy_score(y_test,y_pred1)

In [ ]:
y_pred1=model1.predict_classes(X)
df["prediction"]=y_pred1

In [ ]:
submission=df[["label","prediction"]]
submission

In [ ]:
submission.to_csv("submission.csv")